关于哈希的知识可以自行了解，去leetcode上面刷一点题可以更快掌握有关知识。这里引用百度的一些介绍  

哈希是一种高效的数据结构，用于实现快速的查找、插入和删除操作。其核心思想是通过哈希函数将关键码值（Key）映射到表中的特定位置，从而直接访问记录，避免遍历整个数据集。哈希表通过哈希函数将关键码值映射到表中的位置，从而实现快速访问。哈希函数的设计至关重要，它决定了映射的均匀性和冲突的概率。哈希表的核心优势在于其查找时间复杂度可以达到O(1)，远优于顺序查找的O(N)和平衡树的O(logN)

In [3]:
import open3d.core as o3c
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# 一个简单的例子

In [4]:
# 可以使用o3c.Hashmap()建立哈希表

hashmap = o3c.HashMap(init_capacity=10, # 设置初始容量为10，后续其会自动调整
                      key_dtype=o3c.int64, # 键的数据类型
                      key_element_shape=(1,), # 键的形状
                      value_dtype=o3c.int64, # 值的数据类型
                      value_element_shape=(1,), # 值的形状
                      device=o3c.Device('CPU:0'))

print('新建立的Hashmap',hashmap)

新建立的Hashmap <open3d.cpu.pybind.core.HashMap object at 0x7ee2ab4fee30>


# 嵌入

接下来需要新建两个Tensor以嵌入键值对  
键(key)tensor包含所有键的取值  
值(vals)tensor包含所有值的取值

In [5]:
# 建立7个键值对，每个元素的类型为int64

keys = o3c.Tensor([[100], [200], [400], [800], [300], [200], [100]],
                  dtype=o3c.int64)
vals = o3c.Tensor([[1], [2], [4], [8], [3], [2], [1]],
                  dtype=o3c.int64)


# 使用内置方法insert()将键向量和值向量嵌入至Hashmap中
buf_indices, masks = hashmap.insert(keys, vals)


print('masks: \n', masks)
print('\n inserted keys: \n', keys[masks])

masks: 
 [True True True True True False False]
Tensor[shape={7}, stride={1}, Bool, CPU:0, 0x59b1a3469710]

 inserted keys: 
 [[100],
 [200],
 [400],
 [800],
 [300]]
Tensor[shape={5, 1}, stride={1, 1}, Int64, CPU:0, 0x59b1a362dd40]


这里返回的mask表明是否将一组键值对成功嵌入，值为True表示嵌入成功，False表示跳过。  
如果键tensor有重复的元素(例如keys[0]与keys[-1]同样为100)，只会保证有一个能成功嵌入(此例中keys[0]成功嵌入)，而另一个则会嵌入失败  
嵌入过程是并行的，不能保证哪个重复键是一定会被嵌入的，因此尽量避免键的重复。

buf_indices提供了一个缓冲区索引，通过这个索引可以快速读取键元素与值元素。  
由于过程是并行的，并不保证缓冲区索引与输入索引相同

In [6]:
# 读取已经嵌入的键Tensor与值Tensor
buf_keys = hashmap.key_tensor()
buf_vals = hashmap.value_tensor()

# 读取嵌入成功的键值对的缓冲区索引
buf_indices = buf_indices[masks].to(o3c.int64)
print('buffer indices: \n', buf_indices)  # 该索引并不一定是[0,1,2,3,4] 也有可能出现[0 1 3 4 2]等情况

# 使用buf_indices读取键值
inserted_keys = buf_keys[buf_indices]
inserted_vals = buf_vals[buf_indices]
print('\n inserted keys: \n', inserted_keys)
print('\n inserted values: \n', inserted_vals)

buffer indices: 
 [0 1 2 3 4]
Tensor[shape={5}, stride={1}, Int64, CPU:0, 0x59b1a17326b0]

 inserted keys: 
 [[100],
 [200],
 [400],
 [800],
 [300]]
Tensor[shape={5, 1}, stride={1, 1}, Int64, CPU:0, 0x59b1a3040050]

 inserted values: 
 [[1],
 [2],
 [4],
 [8],
 [3]]
Tensor[shape={5, 1}, stride={1, 1}, Int64, CPU:0, 0x59b1a2dea910]


# 查询

通过键tensor可以查询相应的值

In [14]:
query = o3c.Tensor([[1000], [100], [300], [200], [100], [0]], dtype=o3c.int64)  # 建立用以查询的tensor


buf_indices, masks = hashmap.find(query) # 使用内置方法find()返回buf_indices和masks
valid_keys = query[masks] # 通过mask得到能够查询的键(即query存在于hashmap中的键)

# 通过buf_indices查找对应的值
buf_indices = buf_indices[masks].to(o3c.int64)
valid_vals = hashmap.value_tensor()[buf_indices]

print('能够在hashmap中查询到的键: \n', valid_keys)
print('\n根据query查到的值: \n', valid_vals)

能够在hashmap中查询到的键: 
 [[100],
 [300],
 [200],
 [100]]
Tensor[shape={4, 1}, stride={1, 1}, Int64, CPU:0, 0x59b1a3306a30]

根据query查到的值: 
 [[1],
 [3],
 [2],
 [1]]
Tensor[shape={4, 1}, stride={1, 1}, Int64, CPU:0, 0x59b1a2cbf8d0]


# 消除

In [18]:
# 可以消除hashmap中的键值对

# 需要消除的键
erase_keys = o3c.Tensor([[100], [1000], [100]], dtype=o3c.int64)

# 通过内置方法erase()消除键
masks = hashmap.erase(erase_keys)


print('erase masks:\n', masks)
print('这里只有第一个返回True，因为第一个键[100]存在于hashmap中，因此可以消除。')
print('消除后hashmap就没有属于[100]的键值对了，因此后面的[100]返回False，而[1000]本来就不再hashmap中，所以也返回False\n')

print('\n 消除的键为:\n', erase_keys[masks])

erase masks:
 [True False False]
Tensor[shape={3}, stride={1}, Bool, CPU:0, 0x59b1a3601ca0]
这里只有第一个返回True，因为第一个键[100]存在于hashmap中，因此可以消除。
消除后hashmap就没有属于[100]的键值对了，因此后面的[100]返回False，而[1000]本来就不再hashmap中，所以也返回False


 消除的键为:
 [[100]]
Tensor[shape={1, 1}, stride={1, 1}, Int64, CPU:0, 0x59b1a12e4750]


# 重新排列

如果多次嵌入键值对超过了初始容量，将自动触发重新排列，其中容量会加倍。  
重新排列会改变即缓冲区索引，因此需要更新下游应用程序中的缓冲区索引。

In [20]:
print('目前hashmap的大小:', hashmap.size())
print('目前hashmap的容量:', hashmap.capacity())

# 插入3个新的键值对
keys = o3c.Tensor([[700], [1200], [1500]], dtype=o3c.int64)
vals = o3c.Tensor([[7], [12], [-1]], dtype=o3c.int64)
buf_indices, masks = hashmap.insert(keys, vals)

print('插入3个键值对后hashmap的大小:', hashmap.size())
print('插入3个键值对后hashmap的容量:', hashmap.capacity())

# 再插入4个新的键值对
keys = o3c.Tensor([[1600], [1700], [1800], [1900]] , dtype=o3c.int64)
vals = o3c.Tensor([[16], [17], [18], [19]], dtype=o3c.int64)
buf_indices, masks = hashmap.insert(keys, vals)


print('最终hashmap的大小:', hashmap.size())
print('最终hashmap的容量:', hashmap.capacity())


目前hashmap的大小: 4
目前hashmap的容量: 10
插入3个键值对后hashmap的大小: 7
插入3个键值对后hashmap的容量: 10
最终hashmap的大小: 11
最终hashmap的容量: 20


重新排列的速度较慢，因为它增加了容量，收集了所有有效的键值对，并将它们重新嵌入回哈希映射。如果事先知道容量，可以预先分配一块内存并避免重新排列

In [21]:
hashmap.reserve(100) # 通过reserve改变容量大小
print('hashmap的大小:', hashmap.size())
print('hashmap的容量:', hashmap.capacity())

hashmap的大小: 11
hashmap的容量: 100


# 多值哈希表


在实际应用中，通常需要将坐标映射到复杂的数据结构中（例如体素的位置与其颜色和权重相关联）。这可以通过多值哈希表来实现。

In [8]:
# 新建一个hashmap，注意键tensor的形状和值tensor的形状有所改变
mhashmap = o3c.HashMap(10,
                       key_dtype=o3c.int32,
                       key_element_shape=(3,),
                       value_dtypes=(o3c.uint8, o3c.float32),
                       value_element_shapes=((3,), (1,)))

# 体素坐标，形状为N*3，其中N代表点的个数，3代表体素的坐标（对应key_element_shape），
voxel_coords = o3c.Tensor([[0, 1, 0], [-1, 2, 3], [3, 4, 1]], dtype=o3c.int32)

# 体素的颜色，形状为N*3，其中N代表点的个数，3代表rgb取值（对应value_element_shapes中的(3,)）
voxel_colors = o3c.Tensor([[0, 255, 0], [255, 255, 0], [255, 0, 0]], dtype=o3c.uint8)

# 体素的权重，形状为N*1，其中N代表点的个数，1代表权重（对应value_element_shapes中的(1,)）
voxel_weights = o3c.Tensor([[0.9], [0.1], [0.3]], dtype=o3c.float32)

# 往hashmap嵌入键值对
buf_indices, masks = mhashmap.insert(voxel_coords, (voxel_colors, voxel_weights))
print('masks: \n', masks)
print('\n inserted keys: \n', voxel_coords[masks])

print('\n哈希表不允许重复键，但是支持一个键对应多个值，因此可以通过上述操作实现多值哈希表')

masks: 
 [True True True]
Tensor[shape={3}, stride={1}, Bool, CPU:0, 0x5c38f3a7ed20]

 inserted keys: 
 [[0 1 0],
 [-1 2 3],
 [3 4 1]]
Tensor[shape={3, 3}, stride={3, 1}, Int32, CPU:0, 0x5c38f3cd77b0]

哈希表不允许重复键，但是支持一个键对应多个值，因此可以通过上述操作实现多值哈希表


In [9]:
query_coords = o3c.Tensor([[0, 1, 0]], dtype=o3c.int32)  # 需要查询的体素坐标，即[0,1,0]

buf_indices, masks = mhashmap.find(query_coords)  # 查找[0,1,0]对应的索引位置

valid_keys = query_coords[masks] # 有效的键
buf_indices = buf_indices[masks].to(o3c.int64) # 有效的索引

valid_colors = mhashmap.value_tensor(0)[buf_indices] # [0,1,0]对应的颜色
valid_weights = mhashmap.value_tensor(1)[buf_indices] # [0,1,0]对应的权重

print('查询到的体素坐标:\n', valid_keys)
print('对应的颜色:\n', valid_colors)
print('对应的权重:\n', valid_weights)

查询到的体素坐标:
 [[0 1 0]]
Tensor[shape={1, 3}, stride={3, 1}, Int32, CPU:0, 0x5c38f3cc2860]
对应的颜色:
 [[0 255 0]]
Tensor[shape={1, 3}, stride={3, 1}, UInt8, CPU:0, 0x5c38f2e81bf0]
对应的权重:
 [[0.9]]
Tensor[shape={1, 1}, stride={1, 1}, Float32, CPU:0, 0x5c38f319b7a0]


# 哈希set

哈希set是一个简化的哈希映射，并不关心值。它将大部分操作保留在哈希映射中，对于删除重复项很有用。

In [28]:
# 新建一个hashset
hashset = o3c.HashSet(10,
                      key_dtype=o3c.int64,
                      key_element_shape=(1,))

# 新建一个键tensor，并嵌入hashset
keys = o3c.Tensor([1, 3, 5, 7, 5, 3, 1], dtype=o3c.int64).reshape((-1, 1))
buf_indices , mask = hashset.insert(keys)

print('所有的有效键：', hashset.key_tensor()[mask] )

# 再新建一个键tensor，并嵌入hashset
keys = o3c.Tensor([5, 7, 9, 11], dtype=o3c.int64).reshape((-1, 1))
buf_indices1 , mask1 = hashset.insert(keys)

# 获取有效键的缓冲区索引
active_buf_indices = hashset.active_buf_indices().to(o3c.int64)
active_keys = hashset.key_tensor()[active_buf_indices]

print('\n第二次嵌入后所有有效的键：', active_keys )

所有的有效键： [[1],
 [3],
 [5],
 [7]]
Tensor[shape={4, 1}, stride={1, 1}, Int64, CPU:0, 0x5c38f285f350]

第二次嵌入后所有有效的键： [[5],
 [9],
 [1],
 [3],
 [11],
 [7]]
Tensor[shape={6, 1}, stride={1, 1}, Int64, CPU:0, 0x5c38f3ab6e90]


# 有效的键

In [55]:
# 新建一个hashmap
hashmap = o3c.HashMap(init_capacity=5,
                      key_dtype=o3c.int64, 
                      key_element_shape=(1,), 
                      value_dtype=o3c.int64, 
                      value_element_shape=(1,))

print('键的取值',hashmap.key_tensor())
print('\n由于还未插入键值对，所以键的取值根据初始容量是随机生成的')

键的取值 [[101417873079601],
 [0],
 [4294967295],
 [6845471640740253763],
 [23437470193567828]]
Tensor[shape={5, 1}, stride={1, 1}, Int64, CPU:0, 0x5c38f36e19e0]

由于还未插入键值对，所以键的取值根据初始容量是随机生成的


In [56]:
# 插入一组键值对
keys = o3c.Tensor([[100], [200]],dtype=o3c.int64)
vals = o3c.Tensor([[1], [2]], dtype=o3c.int64)
buf_indices, masks = hashmap.insert(keys, vals)
print('嵌入一组键值对后，键的取值',hashmap.key_tensor())

嵌入一组键值对后，键的取值 [[100],
 [200],
 [4294967295],
 [6845471640740253763],
 [23437470193567828]]
Tensor[shape={5, 1}, stride={1, 1}, Int64, CPU:0, 0x5c38f36e19e0]


由上可以看出，即使插入一组键值后，如果没有填满初始容量，那么键的取值就包含随机数，这时候就需要一种方法找到有效的键(即我们自己定义的键而非随机生成)

In [57]:
# 使用内置方法active_buf_indices()可以返回有效键所处的缓冲区索引
active_buf_indices = hashmap.active_buf_indices().to(o3c.int64)

# 通过返回的缓冲区索引可以找到我们自己定义的键值对
active_keys = hashmap.key_tensor()[active_buf_indices]
print('有效的键:\n', active_keys)

active_vals = hashmap.value_tensor()[active_buf_indices]
print('有效的值:\n', active_vals)

有效的键:
 [[100],
 [200]]
Tensor[shape={2, 1}, stride={1, 1}, Int64, CPU:0, 0x5c38f3ccbd70]
有效的值:
 [[1],
 [2]]
Tensor[shape={2, 1}, stride={1, 1}, Int64, CPU:0, 0x5c38f396ba40]
